In [1]:
import os
import re
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import tqdm
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [2]:
# === CONFIG ===
DOCS_PATH = "../1.CollectingDocuments/data_clean"
LINK_FILE = "../1.CollectingDocuments/extracted_urls.csv"

In [3]:
# === 1️⃣ Đọc dữ liệu và tiền xử lý ===
def clean_text(text):
    # Loại bỏ ký tự đặc biệt, giữ lại tiếng Việt có dấu
    text = re.sub(r"http\S+", "", text)  # bỏ URL
    text = re.sub(r"[^a-zA-ZÀ-ỹ0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.lower()

def load_and_preprocess_docs(folder_path):
    docs = {}
    for fname in os.listdir(folder_path):
        if fname.endswith(".txt"):
            with open(os.path.join(folder_path, fname), "r", encoding="utf-8") as f:
                raw = f.read()
            clean = clean_text(raw)
            tokens = ViTokenizer.tokenize(clean)
            docs[fname] = tokens
    return docs

print("🔄 Đang tải và xử lý tài liệu...")
docs = load_and_preprocess_docs(DOCS_PATH)
print(f"✅ Đã xử lý {len(docs)} tài liệu.\n")

🔄 Đang tải và xử lý tài liệu...
✅ Đã xử lý 406 tài liệu.



In [4]:
# === 2️⃣ Phân tích thống kê cơ bản ===
doc_stats = []
for fname, text in docs.items():
    tokens = text.split()
    total_words = len(tokens)
    unique_words = len(set(tokens))
    top_words = [w for w, _ in Counter(tokens).most_common(10)]
    doc_stats.append({
        "document": fname,
        "total_words": total_words,
        "unique_words": unique_words,
        "top_words": ", ".join(top_words)
    })

df_stats = pd.DataFrame(doc_stats)
print("📊 Thống kê cơ bản:\n", df_stats.head(), "\n")

# === 3️⃣ TF-IDF cho toàn bộ tập tài liệu ===
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(docs.values())
feature_names = vectorizer.get_feature_names_out()

📊 Thống kê cơ bản:
                                             document  total_words  \
0  15_ia_iem_du_lich_Buon_Ma_Thuot_ep_quen_loi_ve...         1807   
1  3_ngoi_lang_co_nep_minh_giua_long_cao_nguyen_a...          952   
2               ao_Binh_Ba__Wikipedia_tieng_Viet.txt          483   
3  ao_Hon_Khoai_o_Ca_Mau_-_vung_bien_va_dai_at_ph...         2553   
4                    a_Lat__Wikipedia_tieng_Viet.txt        15349   

   unique_words                                          top_words  
0           646  khách_sạn, buôn, du_lịch, của, là, thuột, ma, ...  
1           503  của, làng, lô, những, ngôi, đá, khách, với, là...  
2           252  bình, ba, đảo, là, ngày, tháng, năm, 2013, bãi...  
3           749  du_lịch, đảo, châu, hòn, bình, tour, khoai, ni...  
4          2646  năm, và, lạt, đà, tháng, ngày, của, thành_phố,...   



In [5]:
# Trích 5 từ có TF-IDF cao nhất cho mỗi doc
def top_tfidf_words(row, features, top_n=5):
    idx = row.nonzero()[1]
    scores = zip(idx, [row[0, i] for i in idx])
    sorted_words = sorted(scores, key=lambda x: x[1], reverse=True)[:top_n]
    return ", ".join([features[i] for i, _ in sorted_words])

top_tfidf = [top_tfidf_words(tfidf_matrix[i], feature_names) for i in range(len(docs))]
df_stats["top_tfidf"] = top_tfidf

In [6]:
df_stats.head()

,document,total_words,unique_words,top_words,top_tfidf
0,15_ia_iem_du_lich_Buon_Ma_Thuot_ep_quen_loi_ve...,1807,646,"khách_sạn, buôn, du_lịch, của, là, thuột, ma, ...","buôn, st, thuột, ma, khách_sạn"
1,3_ngoi_lang_co_nep_minh_giua_long_cao_nguyen_a...,952,503,"của, làng, lô, những, ngôi, đá, khách, với, là...","lô, làng, lũng, cổ_kính, tường"
2,ao_Binh_Ba__Wikipedia_tieng_Viet.txt,483,252,"bình, ba, đảo, là, ngày, tháng, năm, 2013, bãi...","2013, đảo, bình, ba, truy_cập"
3,ao_Hon_Khoai_o_Ca_Mau_-_vung_bien_va_dai_at_ph...,2553,749,"du_lịch, đảo, châu, hòn, bình, tour, khoai, ni...","khoai, đảo, hòn, du_lịch, châu"
4,a_Lat__Wikipedia_tieng_Viet.txt,15349,2646,"năm, và, lạt, đà, tháng, ngày, của, thành_phố,...","lạt, trần_sỹ, đà, tr, năm"


In [ ]:
# # === 4️⃣ Phân tích liên kết & tính PageRank ===
# if os.path.exists(LINK_FILE):
#     links = pd.read_csv(LINK_FILE)
#     G = nx.DiGraph()
#     for _, row in links.iterrows():
#         src, tgt = row["source_file"], row["target"]
#         if src in docs and tgt in docs:
#             G.add_edge(src, tgt)
    
#     # Tính PageRank (độ phổ biến / ảnh hưởng)
#     pagerank_scores = nx.pagerank(G, alpha=0.85)
#     df_stats["pagerank"] = df_stats["document"].map(pagerank_scores).fillna(0)
# else:
#     df_stats["pagerank"] = 0
#     print("⚠️ Không tìm thấy file liên kết, bỏ qua PageRank.\n")

KeyError: 'source'

In [ ]:
# === 5️⃣ Chuẩn hóa dữ liệu tổng hợp ===
df_stats["avg_tfidf"] = tfidf_matrix.mean(axis=1).A1
# df_stats = df_stats.sort_values(ascending=False)

KeyError: 'pagerank'

In [9]:
print("🏁 Kết quả tổng hợp:")
print(df_stats.head())

🏁 Kết quả tổng hợp:
                                            document  total_words  \
0  15_ia_iem_du_lich_Buon_Ma_Thuot_ep_quen_loi_ve...         1807   
1  3_ngoi_lang_co_nep_minh_giua_long_cao_nguyen_a...          952   
2               ao_Binh_Ba__Wikipedia_tieng_Viet.txt          483   
3  ao_Hon_Khoai_o_Ca_Mau_-_vung_bien_va_dai_at_ph...         2553   
4                    a_Lat__Wikipedia_tieng_Viet.txt        15349   

   unique_words                                          top_words  \
0           646  khách_sạn, buôn, du_lịch, của, là, thuột, ma, ...   
1           503  của, làng, lô, những, ngôi, đá, khách, với, là...   
2           252  bình, ba, đảo, là, ngày, tháng, năm, 2013, bãi...   
3           749  du_lịch, đảo, châu, hòn, bình, tour, khoai, ni...   
4          2646  năm, và, lạt, đà, tháng, ngày, của, thành_phố,...   

                        top_tfidf  avg_tfidf  
0  buôn, st, thuột, ma, khách_sạn   0.002573  
1  lô, làng, lũng, cổ_kính, tường   0.002975  
2  

In [10]:
# === 6️⃣ Xuất ra file tổng hợp ===
os.makedirs("output", exist_ok=True)
df_stats.to_csv("output/doc_analysis.csv", index=False, encoding="utf-8-sig")
print("\n💾 Đã lưu kết quả vào output/doc_analysis.csv")


💾 Đã lưu kết quả vào output/doc_analysis.csv
